In [1]:
import pandas as pd

In [3]:
df_genotype = pd.read_csv("../data/EE_015/EE_015_genotype.csv.gz", sep=";", compression="gzip")
df_default = pd.read_csv("../data/EE_015/EE_015_default.csv.gz", sep=";", compression="gzip")

In [5]:
df_genotype.select_dtypes(object).columns

Index(['ACMG_class', 'ACMG_coding_impact', 'ACMG_gene', 'ACMG_rules',
       'ACMG_transcript', 'AMP_matches', 'AMP_rules', 'AMP_tier',
       'AS_FilterStatus', 'AS_SB_TABLE', 'CGDinheritance', 'Gene',
       'MutationTaster_pred', 'MutationTaster_score', 'RU', 'SIFT_score',
       'coding_impact', 'cosmicFathMMPrediction', 'function', 'hgvs'],
      dtype='object')

In [7]:
df_genotype["ClinVarClass"].unique()

array([  0,   2,   1,   3,   6, 108, 112, 109,   5, 113, 110], dtype=int64)

In [3]:
df = pd.concat([df_default.drop("Unnamed: 0", axis=1), df_genotype.drop("Unnamed: 0", axis=1)], axis=1, ignore_index=False)

In [36]:
object_columns = df.select_dtypes(object).columns

# i = 3
# for col in object_columns[i:i+1]:
#     # print(f"Column {col}: {df[col].unique()}")
#     print(col)
#     unique = df[col].unique()
#     print(len(unique))
#     # print([x for x in unique if len(x.split(";"))>1])

#     for unique_val in unique:
#         print(unique_val)

In [37]:
# df[[c for c in df.columns if c not in df.select_dtypes(object).columns]]

In [6]:
# sklearn.preprocessing.OneHotEncoder

filters = ["PASS", "SB", "UM", "clustered_events", "fragment", "germline", "haplotype", "multiallelic", "slippage"]
filter_cols = ["FILTER_" + f for f in filters]
df["FILTER"].astype(str).str.split(";").iloc[-6]

['PASS', 'clustered_events', 'haplotype']

In [7]:
functions = ["0", "NMD", "3'utr", "5'utr", "3'flank", "5'flank", "coding", "non-coding%40exon", "intronic", "splicing", "splicing-ACMG"]
function_cols = ["function_" + f for f in filters]

uniq = set([])
for x in df["function"].astype(str).str.split(","):
    for y in x:
        if y not in uniq:
            uniq.add(y)

uniq

{'0',
 "3'flank",
 "3'utr",
 "5'flank",
 "5'utr",
 'NMD',
 'coding',
 'intronic',
 'non-coding%40exon',
 'splicing',
 'splicing-ACMG'}

In [8]:
object_columns

Index(['#CHROM', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'ACMG_class',
       'ACMG_coding_impact', 'ACMG_gene', 'ACMG_rules', 'ACMG_transcript',
       'AMP_matches', 'AMP_rules', 'AMP_tier', 'AS_FilterStatus',
       'AS_SB_TABLE', 'CGDinheritance', 'Gene', 'MutationTaster_pred',
       'MutationTaster_score', 'RU', 'SIFT_score', 'coding_impact',
       'cosmicFathMMPrediction', 'function', 'hgvs'],
      dtype='object')

### Object columns:
- #CHROM - no arrays
- ID - id
- REF, ALT - no arrays
- QUAL - binary 1 or .
- FILTER - ["PASS", "SB", "UM", "clustered_events", "fragment", "germline", "haplotype", "multiallelic", "slippage"]
- ACMG_class - no arrays
- ACMG_coding_impact - no arrays
- ACMG_gene - no arrays
- ACMG_rules - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4544753/ I will leave this data unseparated for now 
- ACMG_transcript - no arrays
- AMP_matches - no arrays
- AMP_rules - https://varsome.com/about/resources/somatic-implementation/ I will leave this data unseparated for now
- AMP_tier - no arrays
- AS_FilterStatus - https://docs.varsome.com/en/vcf-attributes-explained I will leave this data unseparated for now
- AS_SB_TABLE - https://docs.varsome.com/en/vcf-attributes-explained I will leave this data unseparated for now
- CGDinheritance - I couldn't find good info, looks separable, left for now
- Gene - gene symbols - might be separable, maybe not worth is if a lot of unique values, check relation to ACMG (target) class
- MutationTaster_pred - I don't know which of the symbols represent which type and why there is a number of them, %3B is ';', left unseparated for now. https://www.mutationtaster.org/info/documentation.html. MutationTaster predicts an alteration as one of four possible types:
    - disease causing - i.e. probably deleterious
    - disease causing automatic - i.e. known to be deleterious, see section dbSNP / TGP / ClinVar / HGMD for details
    - polymorphism - i.e. probably harmless
    - polymorphism automatic - i.e. known to be harmless, see section dbSNP / TGP / ClinVar / HGMD for details
- MutationTaster_score - respective score (certainty?)
- RU - no arrays
- SIFT_score - https://ionreporter.thermofisher.com/ionreporter/help/GUID-2097F236-C8A2-4E67-862D-0FB5875979AC.html, separable but I don't know how
- coding_impact - https://www.ebi.ac.uk/training/online/courses/human-genetic-variation-introduction/what-is-genetic-variation/what-effect-do-variants-in-coding-regions-have/ seems like there are 3 important classes, couldn't find info on the other, leaving unseparated for now
- cosmicFathMMPrediction - no arrays - also target
- function - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2872880/ - ["0", "NMD", "3'utr", "5'utr", "3'flank", "5'flank", "coding", "non-coding%40exon", "intronic", "splicing", "splicing-ACMG"]
- hgvs - https://varnomen.hgvs.org/recommendations/general/ - again I don't know how to separate this well

## Analyzing additional columns in EE_069

In [9]:
df1 = pd.read_csv("data/EE_050/EE_050_default.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("data/EE_050/EE_050_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)

df_50 = pd.concat([df1, df2], ignore_index=False, axis=1)

df1 = pd.read_csv("data/EE_069/EE_069_default.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("data/EE_069/EE_069_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)

df_69 = pd.concat([df1, df2], ignore_index=False, axis=1)

C:\Users\barte\AppData\Local\Temp\ipykernel_32944\4088336807.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("data/EE_069/EE_069_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)


In [10]:
diff_cols_50 = set(df_50.columns).difference(set(df_69.columns))
diff_cols_50

{'AMP_matches',
 'AMP_rules',
 'AMP_score',
 'AMP_tier',
 'AMP_total_samples',
 'AS_FilterStatus',
 'AS_SB_TABLE',
 'ECNT',
 'GERMQ',
 'MBQ',
 'MFRL',
 'MMQ',
 'MPOS',
 'POPAF',
 'RPA',
 'RU',
 'STR',
 'STRQ',
 'TLOD',
 'cosmicFathMMPrediction',
 'cosmicFathMMScore'}

In [11]:
diff_cols_69 = set(df_69.columns).difference(set(df_50.columns))
diff_cols_69

{'AC',
 'AF',
 'AN',
 'BaseQRankSum',
 'ClippingRankSum',
 'ExcessHet',
 'FS',
 'MLEAC',
 'MLEAF',
 'MQ',
 'MQRankSum',
 'QD',
 'ReadPosRankSum',
 'SOR'}

In [38]:
# df_69[list(diff_cols_69)]

In [13]:
object_columns = df_69[list(diff_cols_69)].select_dtypes(object).columns

object_columns

Index(['BaseQRankSum', 'MQRankSum', 'ClippingRankSum', 'ReadPosRankSum'], dtype='object')

In [14]:
i = 2
for col in object_columns[i:i+1]:
    # print(f"Column {col}: {df[col].unique()}")
    print(col)
    unique = df_69[col].unique()
    print(len(unique))
    # print([x for x in unique if len(x.split(";"))>1])

    for unique_val in unique:
        print(unique_val)

ClippingRankSum
2
0
.


- ReadPosRankSum - no arrays looks like float + nan
- ClippingRankSum - no arrays
- MQRankSum - no arrays - float and nan
- BaseQRankSum - same

## Analyzing allel and uncsv read difference

In [15]:
import pandas as pd

In [16]:
df1 = pd.read_csv("data/EE_015/EE_015_default.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("data/EE_015/EE_015_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)

df_15 = pd.concat([df1, df2], axis=1, ignore_index=False)

In [39]:
# df_15

In [18]:
df1 = pd.read_csv("data/EE_050/EE_050_default.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("data/EE_050/EE_050_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)

df_50 = pd.concat([df1, df2], axis=1, ignore_index=False)

In [40]:
# df_50

In [20]:
df1 = pd.read_csv("data/EE_069/EE_069_default.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("data/EE_069/EE_069_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)

df_69 = pd.concat([df1, df2], axis=1, ignore_index=False)

C:\Users\barte\AppData\Local\Temp\ipykernel_32944\3002690269.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("data/EE_069/EE_069_genotype.csv.gz", sep=";", compression="gzip").drop("Unnamed: 0", axis=1)


In [21]:
# df_69

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,AC,ACMG_class,ACMG_coding_impact,...,gnomadExomes_AF_ethnic,gnomadExomes_AN,gnomadGenomesAC,gnomadGenomesAN,gnomadGenomesEthnic_AC_Hom,gnomadGenomes_AC_Hemi,gnomadGenomes_AC_Hom,gnomadGenomes_AF,gnomadGenomes_AF_ethnic,hgvs
0,chr1,14653,rs62635297,C,T,331.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,31329,79992,420,0,31329,0.391652,0.447103,0
1,chr1,17594,rs377698370,C,T,109.77,PASS,1,Benign,non%40coding,...,0.0,0,3276,132414,0,0,3276,0.024741,0.007104,0
2,chr1,17614,rs201057270,G,A,258.77,PASS,1,Benign,non%40coding,...,0.0,0,6270,126212,0,0,6270,0.049678,0.056625,0
3,chr1,17697,rs71260069,G,C,224.77,PASS,1,Benign,non%40coding,...,0.0,0,17539,116050,2,0,17539,0.151133,0.178196,0
4,chr1,17722,rs376731495,A,G,313.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,2836,123488,0,0,2836,0.022966,0.027542,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140169,chrY,56856571,rs9988391,G,A,293.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,0,0,0,0,0,0.000000,0.000000,0
140170,chrY,56856581,rs9320083,G,A,750.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,2,135,0,0,2,0.014815,0.000000,0
140171,chrY,56856587,rs7067511,A,C,1164.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,123,123,0,0,123,1.000000,1.000000,0
140172,chrY,56856668,rs5013140,A,G,467.77,PASS,1,Uncertain%40Significance,non%40coding,...,0.0,0,101,103,0,0,101,0.980583,1.000000,0


In [22]:
df_csv = pd.concat([df_15, df_50, df_69], axis=0)

In [23]:
df_csv

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,ACMG_class,ACMG_coding_impact,ACMG_gene,...,ClippingRankSum,ExcessHet,FS,MLEAC,MLEAF,MQ,MQRankSum,QD,ReadPosRankSum,SOR
0,chr1,15820,rs2691315,G,T,.,PASS,Uncertain%40Significance,non%40coding,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,17385,rs201535981,G,A,.,PASS,Uncertain%40Significance,non%40coding,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,17697,rs71260069,G,C,.,PASS,Benign,non%40coding,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,133129,rs367730352,G,A,.,PASS,Uncertain%40Significance,non%40coding,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,183629,rs71267774,G,A,.,PASS,Benign,non%40coding,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140169,chrY,56856571,rs9988391,G,A,293.77,PASS,Uncertain%40Significance,non%40coding,0,...,0,3.0103,8.315,1.0,0.5,41.25,-0.431,5.76,-1.02,0.419
140170,chrY,56856581,rs9320083,G,A,750.77,PASS,Uncertain%40Significance,non%40coding,0,...,0,3.0103,2.660,1.0,0.5,41.23,-2.005,16.68,-0.205,0.361
140171,chrY,56856587,rs7067511,A,C,1164.77,PASS,Uncertain%40Significance,non%40coding,0,...,0,3.0103,0.000,1.0,0.5,40.19,-3.699,31.48,-1.691,0.530
140172,chrY,56856668,rs5013140,A,G,467.77,PASS,Uncertain%40Significance,non%40coding,0,...,0,3.0103,0.000,1.0,0.5,44.02,-1.952,33.41,0.183,2.985


In [24]:
import allel

df_allel_15 = allel.vcf_to_dataframe("data/EE_015/EE_015.vcf.gz", ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])
df_allel_50 = allel.vcf_to_dataframe("data/EE_050/EE_050.vcf.gz", ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])
df_allel_69 = allel.vcf_to_dataframe("data/EE_069/EE_069.vcf.gz", ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"])

df_allel = pd.concat([df_allel_15, df_allel_50, df_allel_69], axis=0)

c:\Users\barte\anaconda3\envs\pathogen\Lib\site-packages\allel\io\vcf_read.py:1240: UserWarning: '#CHROM' INFO header not found
  warnings.warn('%r INFO header not found' % name)
c:\Users\barte\anaconda3\envs\pathogen\Lib\site-packages\allel\io\vcf_read.py:1454: UserWarning: no type for field 'variants/#CHROM', assuming object
  warnings.warn('no type for field %r, assuming %s' % (f, normed_types[f]))
c:\Users\barte\anaconda3\envs\pathogen\Lib\site-packages\allel\io\vcf_read.py:1564: UserWarning: no number for field 'variants/#CHROM', assuming 1
  warnings.warn('no number for field %r, assuming 1' % f)


In [25]:
df_allel.drop("CSQ", inplace=True, axis=1)

In [26]:
df_allel

,#CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS,FILTER_SB,...,MLEAC_2,MLEAC_3,MLEAF_1,MLEAF_2,MLEAF_3,MQ,MQRankSum,QD,ReadPosRankSum,SOR
0,NaN,15820,rs2691315,G,T,NaN,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,17385,rs201535981,G,A,NaN,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,17697,rs71260069,G,C,NaN,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,133129,rs367730352,G,A,NaN,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,183629,rs71267774,G,A,NaN,NaN,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9097,NaN,56856571,rs9988391,G,A,NaN,NaN,293.769989,True,False,...,-1.0,-1.0,0.5,NaN,NaN,41.250000,-0.431,5.76,-1.020,0.419
9098,NaN,56856581,rs9320083,G,A,NaN,NaN,750.770020,True,False,...,-1.0,-1.0,0.5,NaN,NaN,41.230000,-2.005,16.68,-0.205,0.361
9099,NaN,56856587,rs7067511,A,C,NaN,NaN,1164.770020,True,False,...,-1.0,-1.0,0.5,NaN,NaN,40.189999,-3.699,31.48,-1.691,0.530
9100,NaN,56856668,rs5013140,A,G,NaN,NaN,467.769989,True,False,...,-1.0,-1.0,0.5,NaN,NaN,44.020000,-1.952,33.41,0.183,2.985


In [27]:
print(set(df_15.columns).symmetric_difference(set(df_50.columns)))
print(set(df_15.columns).symmetric_difference(set(df_69.columns)))
print(set(df_50.columns).symmetric_difference(set(df_69.columns)))

set()
{'STRQ', 'ClippingRankSum', 'GERMQ', 'MBQ', 'POPAF', 'ReadPosRankSum', 'AMP_total_samples', 'ExcessHet', 'MLEAF', 'AMP_tier', 'MQRankSum', 'RPA', 'MPOS', 'TLOD', 'QD', 'STR', 'cosmicFathMMPrediction', 'AMP_score', 'AN', 'SOR', 'MQ', 'AS_FilterStatus', 'ECNT', 'AMP_rules', 'BaseQRankSum', 'AS_SB_TABLE', 'AF', 'FS', 'MLEAC', 'MMQ', 'cosmicFathMMScore', 'AMP_matches', 'MFRL', 'RU', 'AC'}
{'STRQ', 'ClippingRankSum', 'GERMQ', 'MBQ', 'POPAF', 'ReadPosRankSum', 'AMP_total_samples', 'ExcessHet', 'MLEAF', 'AMP_tier', 'MQRankSum', 'RPA', 'MPOS', 'TLOD', 'QD', 'STR', 'cosmicFathMMPrediction', 'AMP_score', 'AN', 'SOR', 'MQ', 'AS_FilterStatus', 'ECNT', 'AMP_rules', 'BaseQRankSum', 'AS_SB_TABLE', 'AF', 'FS', 'MLEAC', 'MMQ', 'cosmicFathMMScore', 'AMP_matches', 'MFRL', 'RU', 'AC'}


In [28]:
allel_csv_difference = set(df_allel.columns).difference(set(df_csv.columns))
allel_csv_difference

{'AC_1',
 'AC_2',
 'AC_3',
 'AF_1',
 'AF_2',
 'AF_3',
 'ALT_1',
 'ALT_2',
 'ALT_3',
 'FILTER_FAIL',
 'FILTER_PASS',
 'FILTER_SB',
 'FILTER_UM',
 'FILTER_base_qual',
 'FILTER_clustered_events',
 'FILTER_fragment',
 'FILTER_germline',
 'FILTER_haplotype',
 'FILTER_map_qual',
 'FILTER_multiallelic',
 'FILTER_slippage',
 'FILTER_strand_bias',
 'FILTER_weak_evidence',
 'MBQ_1',
 'MBQ_2',
 'MBQ_3',
 'MBQ_4',
 'MFRL_1',
 'MFRL_2',
 'MFRL_3',
 'MFRL_4',
 'MLEAC_1',
 'MLEAC_2',
 'MLEAC_3',
 'MLEAF_1',
 'MLEAF_2',
 'MLEAF_3',
 'MMQ_1',
 'MMQ_2',
 'MMQ_3',
 'MMQ_4',
 'MPOS_1',
 'MPOS_2',
 'MPOS_3',
 'POPAF_1',
 'POPAF_2',
 'POPAF_3',
 'RPA_1',
 'RPA_2',
 'RPA_3',
 'RPA_4',
 'TLOD_1',
 'TLOD_2',
 'TLOD_3',
 'gnomadExomesEthnic_AC_Hom_1',
 'gnomadExomesEthnic_AC_Hom_2',
 'gnomadExomesEthnic_AC_Hom_3',
 'gnomadExomes_AC_1',
 'gnomadExomes_AC_2',
 'gnomadExomes_AC_3',
 'gnomadExomes_AC_Hemi_1',
 'gnomadExomes_AC_Hemi_2',
 'gnomadExomes_AC_Hemi_3',
 'gnomadExomes_AC_Hom_1',
 'gnomadExomes_AC_Hom_2',
 

In [29]:
df_allel["MBQ_3"].unique()

array([-1., nan])

In [30]:
df_csv["AC"].unique()

array([nan,  1.,  2.])

#### Columns split (seemingly unnecessarily) into 3/4 columns in allel:
- AC, MBQ, FILTER, MLEAC, MLEAF, MFRL, MPOS, POPAF, RPA, TLOD
- gnomadExomesEthnic_AC_xxx, gnomadExomesEthnic_AF_xxx, NOT gnomadGenomes_AF_ethnic 